In [1]:
import requests
import pandas as pd
import re
import bs4

In [2]:
import requests

url = 'https://www.ebi.ac.uk/merops/cgi-bin/smi_index'

page = requests.get(url).content.decode()

In [4]:
response = requests.get(url)
content = response.content
page = content.decode()

<Response [200]>

In [ ]:
import bs4

soup = bs4.BeautifulSoup(content, 'lxml') # needs to pass through content (bytes) not page (string)

In [16]:
table = soup.find('table')

In [20]:
table_rows = table.find_all('tr')
type(table_rows)

bs4.element.ResultSet

In [ ]:
for row in table_rows:
    td = row.find_all('td')
    for data in td:
        print(data.text)

In [22]:
# pandas 

tables = pd.read_html(url)
tables

[                                                      (  \
 0     (+)-2-[4-[(-1-acetimidoyl-4-piperidinyl)oxy]- ...   
 1     (2-benzyl-3-mercaptopropanoyl)-L-alanylglycina...   
 2     (2S)-5-amino-2-[(1-n-propyl-1H-imidazol-4-yl)m...   
 3     (3R)-amino-(2S)-hydroxyheptanoyl-l-Ala-l-Leu-l...   
 4     (3S,4S)-3-amino-4-mercapto-6-phenyl-hexane-1-s...   
 ...                                                 ...   
 2814                                           zVLL-CHO   
 2815                                            ZY15557   
 2816                                            ZYDPLA1   
 2817                                                  {   
 2818       4-[(4-arsonophenyl)methyl]phenylarsonic acid   
 
                                                     (.1  (.2  
 0                                              DX-9065a  NaN  
 1     (2-benzyl-3-mercaptopropanoyl)-L-alanylglycina...  NaN  
 2                                             UK-396082  NaN  
 3        small-molecu

# Zillow 

In [54]:
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

In [102]:
url = 'https://www.zillow.com/homes/carlsbad_rb/'

page = requests.get(url, headers = req_headers).content
 
soup = BeautifulSoup(page, 'html.parser') # use html.pasrer instead of lxml
soup.find_all(class_ = 'list-card-addr') # gets a list of all othe tags that fit the search term

[<address class="list-card-addr">3460 Harding St, Carlsbad, CA 92008</address>,
 <address class="list-card-addr">1676 Corte Orchidia, Carlsbad, CA 92011</address>,
 <address class="list-card-addr">3307 Rancho Carrizo, Carlsbad, CA 92009</address>,
 <address class="list-card-addr">2630 Pirineos Way UNIT 11, Carlsbad, CA 92009</address>,
 <address class="list-card-addr">3131 Avenida Topanga, Carlsbad, CA 92009</address>,
 <address class="list-card-addr">2501 Navarra Dr UNIT 124, Carlsbad, CA 92009</address>,
 <address class="list-card-addr">7266 San Luis St, Carlsbad, CA 92011</address>,
 <address class="list-card-addr">3482 Rich Field Dr, Carlsbad, CA 92010</address>,
 <address class="list-card-addr">3129 Avenida Topanga, Carlsbad, CA 92009</address>]

In [103]:
type(soup.find_all(class_ = 'list-card-addr')[0]) # what is the type of each of these elements

bs4.element.Tag

In [101]:
addresses = soup.find_all(class_ = 'list-card-addr') # get addreses
for address in addresses: # get addreses
    print(address.text)

3460 Harding St, Carlsbad, CA 92008
1676 Corte Orchidia, Carlsbad, CA 92011
3307 Rancho Carrizo, Carlsbad, CA 92009
2630 Pirineos Way UNIT 11, Carlsbad, CA 92009
3131 Avenida Topanga, Carlsbad, CA 92009
2501 Navarra Dr UNIT 124, Carlsbad, CA 92009
7266 San Luis St, Carlsbad, CA 92011
3482 Rich Field Dr, Carlsbad, CA 92010
3129 Avenida Topanga, Carlsbad, CA 92009


In [99]:
prices = soup.find_all(class_ = 'list-card-price') # get list of prices tags
for price in prices: # for every price tag, get text (content within tags) 
    print(price.text)

$1,295,000
$1,575,000
$1,149,999
$559,900
$537,000
$639,000
$219,000
$1,290,000
$615,000


# this is some higher level python that should make sense to you

In [112]:
def get_text(soup, class_searchterm): # makes a general function to save me time
    results = soup.find_all(class_ = class_searchterm)
    return [result.text for result in results]

prices = get_text(soup, 'list-card-price')
addresses = get_text(soup, 'list-card-addr')



In [117]:
# use a dictionary <3

data = {'address': [], 'prices': []}
for index, address in enumerate(addresses):
    data['address'].append(address)
    data['prices'].append(prices[index])
    
data

{'address': ['3460 Harding St, Carlsbad, CA 92008',
  '1676 Corte Orchidia, Carlsbad, CA 92011',
  '3307 Rancho Carrizo, Carlsbad, CA 92009',
  '2630 Pirineos Way UNIT 11, Carlsbad, CA 92009',
  '3131 Avenida Topanga, Carlsbad, CA 92009',
  '2501 Navarra Dr UNIT 124, Carlsbad, CA 92009',
  '7266 San Luis St, Carlsbad, CA 92011',
  '3482 Rich Field Dr, Carlsbad, CA 92010',
  '3129 Avenida Topanga, Carlsbad, CA 92009'],
 'prices': ['$1,295,000',
  '$1,575,000',
  '$1,149,999',
  '$559,900',
  '$537,000',
  '$639,000',
  '$219,000',
  '$1,290,000',
  '$615,000']}

In [119]:
dataframe = pd.DataFrame.from_dict(data)
dataframe

,address,prices
0,"3460 Harding St, Carlsbad, CA 92008","$1,295,000"
1,"1676 Corte Orchidia, Carlsbad, CA 92011","$1,575,000"
2,"3307 Rancho Carrizo, Carlsbad, CA 92009","$1,149,999"
3,"2630 Pirineos Way UNIT 11, Carlsbad, CA 92009","$559,900"
4,"3131 Avenida Topanga, Carlsbad, CA 92009","$537,000"
5,"2501 Navarra Dr UNIT 124, Carlsbad, CA 92009","$639,000"
6,"7266 San Luis St, Carlsbad, CA 92011","$219,000"
7,"3482 Rich Field Dr, Carlsbad, CA 92010","$1,290,000"
8,"3129 Avenida Topanga, Carlsbad, CA 92009","$615,000"


In [120]:
dataframe.to_csv('zillow_carlsbad.csv')